In [4]:
import tensorflow as tf
print(tf.__version__)

ImportError: Traceback (most recent call last):
  File "C:\Users\LENOVO\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

In [ ]:
print(tf.__version__)

In [ ]:
data = pd.read_csv(r"C:\Users\LENOVO\Documents\Presentation_mini\car_customers web.csv")
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
x = data.drop(['CarModel','Manufacturer','Price','Purchased'], axis=1)
x.head()

In [ ]:
y = data['CarModel']
y.head()

In [ ]:
encode = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_data = encode.fit_transform(x[['BodyType','FuelType','DesiredFeatures']].replace(-1,1))

In [ ]:
scalar = StandardScaler()
num_data = scalar.fit_transform(x[['SafetyRating','Budget','Age','FamilySize']])

In [ ]:
print("Categorical Data Shape:", cat_data.shape)
print("Numerical Data Shape:", num_data.shape)

In [ ]:
combine_x = np.hstack([cat_data, num_data])

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(combine_x, y, test_size=0.2, random_state=42)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# Find unique categories in the target column
unique_classes = data["CarModel"].unique()
print(unique_classes)
print("Number of unique classes:", len(unique_classes))
unique_classes = len(unique_classes)

In [ ]:
print(unique_classes)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(unique_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Convert car model names to numeric labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test) 

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert integer labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded, num_classes=29)
y_test_onehot = to_categorical(y_test_encoded, num_classes=29)


In [ ]:
model.fit(x_train, y_train_onehot, epochs=100, validation_data=(x_test, y_test_onehot))

In [ ]:
model.save("Car_Suggestion_model.h5")

In [ ]:
import joblib
joblib.dump(encode, "encoder.pkl")
joblib.dump(scalar, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

In [ ]:
new_customer = pd.DataFrame({
    'BodyType': ['SUV'],  
    'FuelType': ['diesel'],  
    'SafetyRating': [4],  
    'Budget': [1500000],  
    'DesiredFeatures': ['Sunroofs'],  
    'Age': [20],  
    'FamilySize': [4]
})

In [ ]:
# Load encoders
encoder = joblib.load("encoder.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("Car_Suggestion_model.h5")

In [ ]:
cat_data = encoder.transform(new_customer[['BodyType','FuelType','DesiredFeatures']])
num_data = scaler.transform(new_customer[['SafetyRating','Budget','Age','FamilySize']])

In [ ]:
new = np.hstack([cat_data, num_data])

In [ ]:
softmax_output = model.predict(new)

predicted_index = softmax_output.argmax()

car_classes = ['Sonet', 'Seltos', 'XUV700', 'City', 'Kwid', 'Verna', 'Glanza', 'Ertiga', 'i20',
 'Swift', 'Ignis', 'Wagon R', 'Compass', 'Kiger', 'Venue', 'Magnite', 'Tiago',
 'Innova Crysta', 'Nexon', 'Harrier', 'Alto', 'Celerio', 'Altroz', 'Brezza',
 'Jazz', 'Creta', 'Fortuner', 'Punch', 'Baleno']

# Get the predicted car name
predicted_car = car_classes[predicted_index]

print(f"Suggested Car Model: {predicted_car}")